## 12.1 Description

A signature algorithm is the public-key equivalent of a message authentication code. It consists of three parts:
1. a key generation algorithm, which can be **shared** with other public-key algorithms
2. a signature **generation** algorithm
3. a signature **verification** algorithm

## 12.3 DSA

The Digital Signature Algorithm (DSA) is a US Federal Government standard for digital signatures. It was first proposed by the National Institute of Standards and Technology (NIST) in 1991, to be used in the Digital Signature Standard (DSS).

> note: FIPS 186-5 (2023) the DSA is deprecated

DSA key generation happens in two steps. The first step is a choice of parameters, which can be shared between users. The second step is the generation of public and private keys for a single user.

### Parameter generation

We start by picking an approved cryptographic hash function $H$. We also pick a key length $L$ and a prime length $N$. Next we choose a prime $q$ of length $N$ bits; $N$ must be less than or equal to the length of the hash output. We also pick an $L$-bit prime $p$ such that $p − 1$ is a multiple of $q$.

The last part is the most confusing. We have to find a number $g$ whose *multiplicative order* $\pmod p$ is $q$. The easy way to do this is to set $$g \equiv 2^{(p−1)/q} \pmod p$$

Once we have parameters $(p, q, g)$, they can be shared between users.

### Key generation

First, select a random $x$ with $0 < x < q$. Next, calculate $y$ where $y \equiv g^x \pmod p$. This delivers a public key $(p, q, g, y)$, and private key $x$.

### Signing a message

In order to sign a message, the signer picks a random $k$ between $0$ and $q$.
With $k$ chosen, they then compute the two parts of the signature $r, s$ of the message $m$:
$$r \equiv (g^k \pmod p) \pmod q$$
$$s \equiv k^{-1}(H(m) + xr) \pmod q$$


### Verifying a signature

Verifying the signature is a lot more complex. Given the message $m$ and signature $(r, s)$:
$$w \equiv s^{-1} \pmod q$$
$$u_1 \equiv H(m)w \pmod q$$
$$u_2 \equiv rw \pmod q$$
$$v \equiv (g^{u_1}y^{u_2} \pmod p) \pmod q$$
If the signature is valid that final result $v$ will be equal to $r$, the second part of the signature.

### The trouble with $k$

In particular, the choice of the signature parameter $k$ is critical. DSA's requirements for the $k$ value are a combination of all of these:
- It has to be unique.
- It has to be unpredictable.
- It has to be secret.

Suppose that an attacker sees multiple signatures $(r_i, s_i)$, for different messages $m_i$, all with the same $k$. The attacker picks any two signatures $(r_1, s_1)$ and $(r_2, s_2)$ of messages $m_1$
and $m_2$ respectively. Writing down the equations for $s_1$ and
$s_2$:

$$s_1 \equiv k^{-1}(H(m_1) + xr_1) \pmod q$$
$$s_2 \equiv k^{-1}(H(m_2) + xr_2) \pmod q$$


The attacker can simplify this further: $ r_1 $ and $ r_2 $ must be
equal, following the definition: 

$$
r_i \equiv g^k \pmod{q}
$$

Since the signer is reusing $ k $, and the value of $ r $ only depends on $ k $, all $ r_i $ will be equal. Since the signer is using the same key, $ x $ is equal in the two equations as well.

Subtract the two $s_i$ equations from each other, followed by some other arithmetic manipulations:

$$
\begin{split}
s_1 - s_2 &\equiv k^{-1}(H(m_1) + xr) - k^{-1}(H(m_2) + xr) \pmod q \\
&\equiv k^{-1}((H(m_1) + xr) - (H(m_2) + xr)) \pmod q \\
&\equiv k^{-1}(H(m_1) + xr - H(m_2) - xr) \pmod q \\
&\equiv k^{-1}(H(m_1) - H(m_2)) \pmod q
\end{split}
$$



$$
k \equiv s_1 - s_2 \cdot (H(m_1) - H(m_2))^{-1} \pmod q 
$$

The hash values $H(m_1)$ and $H(m_2)$ are easy to compute. They're not secret: the messages being signed are public. The two values $s_1$ and $s_2$ are part of the signatures the attacker saw. So, the attacker can compute $k$.

$$ s \equiv k^{-1}(H(m) + xr) \pmod q $$
$$ sk \equiv H(m) + xr \pmod q $$
$$ sk - H(m) \equiv xr \pmod q $$
$$ r^{-1}(sk - H(m)) \equiv x \pmod q $$


In short, reusing the $k$ parameter of a DSA signing operation means an attacker recovers the private key $x$.

# Key derivation functions
## 13.1 Description

A key derivation function is a function that derives one or more secret values (the *keys*) from one secret value.
> on the block cipher, other name called key schedule

There are two main categories of key derivation functions, depending on the entropy content of the secret value, which determines how many different possible values the secret value can take.

If the secret value is a user-supplied password, for example, it typically contains very **little entropy**. There are very few values the password will take.

On the other hand, the secret value could also have a **high entropy** content. For example, it could be a shared secret derived from a Diffie-Hellman key agreement protocol, or an API key consisting of cryptographically random bytes (we'll discuss cryptographically secure random number generation in the next chapter).

## 13.6 HKDF

The HKDF, defined in RFC 5869 [KE] and explained in detail in a related paper [Kra10], is a key derivation function designed for *high entropy inputs*, such as shared secrets from a Diffie-Hellman key exchange.

### A closer look at HKDF

HKDF consists of two phases. In the first phase, called the *extraction phase*, a fixed-length key is extracted from the input entropy. In the second phase, called the *expansion phase*, that key is used to produce a number of pseudorandom keys.

#### The extraction phase

The extraction phase is responsible for extracting a small amount of data with a high entropy content from a potentially large amount of data with a smaller entropy density.

#### The expansion phase

In the expansion phase, the random data extracted from the inputs in the extraction phase is expanded into as much data as is required.

In [21]:
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))


def hmac(key, message):
    from hashlib import sha256

    block_size = 64
    if len(key) > block_size:
        key = sha256(key).digest()
    if len(key) < block_size:
        key += b"\x00" * (block_size - len(key))
    o_key_pad = bytes(xor_bytes(b"\x5c" * block_size, key))
    i_key_pad = bytes(xor_bytes(b"\x36" * block_size, key))
    return sha256(o_key_pad + sha256(i_key_pad + message).digest()).digest()


def expand(key, info=""):
    output = ""

    for byte in map(chr, range(256)):
        # Ensure all parts are bytes
        output = output if isinstance(output, bytes) else output.encode()
        info = info if isinstance(info, bytes) else info.encode()
        byte = byte if isinstance(byte, bytes) else byte.encode()
        output = hmac(key, output + info + byte)
        yield output


def get_output(desired_length, key, info=""):

    outputs, current_length = [], 0
    for output in expand(key, info):
        outputs.append(output.hex())
        current_length += len(output.hex())
        if current_length >= desired_length:
            break
    else:
        raise RuntimeError("Desired length too long")
    # Join the decoded strings and slice to the desired length
    return "".join(outputs)[:desired_length]


# Example usage of the HMAC function
key = b"secret_key"
message = b"This is a message"

# Calculate HMAC
hmac_result = hmac(key, message)
print(f"HMAC: {hmac_result.hex()}")

# Example usage of the expand function
key = b"secret_key"
info = b""

# Generate expanded output
expanded_output = list(expand(key, info))
print(
    f"Expanded Output: {[output.hex() for output in expanded_output[:5]]}"
)  # Print first 5 outputs for brevity

# Example usage of the get_output function
desired_length = 64
key = b"secret_key"
info = b""

# Get output of desired length
output = get_output(desired_length, key, info)
print(f"Output of desired length: {output}")

HMAC: 55ac7e647c55ef0552d991f79aa2c7037c6585c0c98aa4c1ac206b1933fc030e
Expanded Output: ['54beac9268a9becff5e1f66b69be4d9240b3035f14cf76505a90fa119bbda7a1', '4c291dcd650b907ff9c3a9972811ed8079c1e4e0d480da422e1a61e8ab31f40b', '7b5f602af9511ee20f74dd7eec1d5f2a3643296304a95335122bd7c6a0dfd46d', '3e516cde8f70b2e6256c4bcf2faa53c977827287924bc96b97d73e45b20f6ecb', '2ea17b9cbfb12dffee32f7581f46a598097229ff6f9c7d8612f0b933fe0c2ee2']
Output of desired length: 54beac9268a9becff5e1f66b69be4d9240b3035f14cf76505a90fa119bbda7a1
